In [3]:
import pandas as pd
import ast


In [4]:
df = pd.read_csv('raw/train.csv')
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [5]:
## Extracts Json objects to lists prioitizing names
features_to_fix=['belongs_to_collection', 'genres', 'production_companies','production_countries','spoken_languages','Keywords']
for feature in features_to_fix:
    df.loc[df[feature].notnull(),feature]=df.loc[df[feature].notnull(),feature].apply(lambda x : ast.literal_eval(x)).apply(lambda x : [y["name"] for y in x])

In [6]:
## Extracts jason objects to dictionaries for cast and crew with 
df.loc[df["cast"].notnull(),"cast"]=df.loc[df["cast"].notnull(),"cast"].apply(lambda x : ast.literal_eval(x))
df.loc[df["crew"].notnull(),"crew"]=df.loc[df["crew"].notnull(),"crew"].apply(lambda x : ast.literal_eval(x))

In [7]:
BinaryMap = {True:0, False:1}
df['hasWebpage']= df.homepage.isna().map(BinaryMap)



In [8]:
allgenres =[]
for genrelist in df.genres:
    try:    
        allgenres.extend(genrelist)
    except(TypeError):   
        ##print(genrelist) Prints verication that what it is failing is missing values
        pass
allgenres = set(allgenres)
print(allgenres)

{'Science Fiction', 'Mystery', 'Comedy', 'Music', 'Fantasy', 'Crime', 'Thriller', 'Romance', 'Horror', 'Family', 'TV Movie', 'War', 'Animation', 'Drama', 'Documentary', 'Action', 'Adventure', 'Western', 'History', 'Foreign'}


In [65]:
##handles missing values for spoken language  based on original value
df.loc[(df['spoken_languages'].isnull()) & (df['original_language']=='ru'),'spoken_languages']= ['Pусский']
df.loc[(df['spoken_languages'].isnull()) & (df['original_language']=='fr'),'spoken_languages']= ['Français']
df.loc[(df['spoken_languages'].isnull()) & (df['original_language']=='en'),'spoken_languages']= ['English']
alllangauges =[]
for languages in df.spoken_languages:
    try:
        alllangauges.extend(languages)
    except:
        alllanguages.append(languages)

alllangauges = set(alllangauges)
alllangauges.remove('')
print(alllangauges)
df.spoken_languages.value_counts()[df.spoken_languages.value_counts() ==1]

{'Esperanto', 'Deutsch', 'తెలుగు', 'Türkçe', 'Afrikaans', 'Norsk', 'Somali', 'ελληνικά', '日本語', 'и', 'ਪੰਜਾਬੀ', 'r', 'с', 'Pусский', 'Magyar', 'Română', 'Català', 'Bahasa indonesia', 'български език', 'Íslenska', 'فارسی', 'isiZulu', 'Polski', 'پښتو', 'suomi', 'у', 'Latin', 'Srpski', '广州话 / 廣州話', 'اردو', 'Eesti', 'ç', 'हिन्दी', 'g', 'h', '한국어/조선말', 'Tiếng Việt', 'Español', 'shqip', 'বাংলা', 'Dansk', 'i', 'a', 'Slovenčina', 'Italiano', 's', 'தமிழ்', 'Gaeilge', 'F', 'қазақ', 'euskera', 'English', 'n', 'ภาษาไทย', 'й', 'No Language', 'Wolof', 'Nederlands', 'العربية', 'l', 'Český', 'Français', 'к', 'עִבְרִית', 'svenska', 'Português', 'Hrvatski', '普通话', 'Український', 'E', 'P', 'Kiswahili'}


[Český]                                                                                   1
[, English, Français, Tiếng Việt]                                                         1
[English, suomi]                                                                          1
[日本語, Français, Pусский, English, العربية]                                                1
[Français, English, Deutsch]                                                              1
[日本語, 普通话, العربية, English]                                                              1
[English, Français, Latin, Español]                                                       1
[Español, English, Deutsch, Nederlands, Français]                                         1
[বাংলা]                                                                                   1
[日本語, English, Français, , Deutsch]                                                       1
[English, ελληνικά, Pусский]                                                    

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 24 columns):
id                       3000 non-null int64
belongs_to_collection    604 non-null object
budget                   3000 non-null int64
genres                   2993 non-null object
homepage                 946 non-null object
imdb_id                  3000 non-null object
original_language        3000 non-null object
original_title           3000 non-null object
overview                 2992 non-null object
popularity               3000 non-null float64
poster_path              2999 non-null object
production_companies     2844 non-null object
production_countries     2945 non-null object
release_date             3000 non-null object
runtime                  2998 non-null float64
spoken_languages         3000 non-null object
status                   3000 non-null object
tagline                  2403 non-null object
title                    3000 non-null object
Keywords             